<a href="https://colab.research.google.com/github/Alex-Devoid/ST-554-Project1/blob/main/Task1/Task1_Project1_Alex_Devoid0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 – Task 1: Grid Search + Gradient Descent

Group members: Emma Martinez, Lanette Tyler, Alex Devoid  
Task 1 author: Alex Devoid  
Course: ST 554 (Spring 2026)

## Introduction


The goal of this notebook is to implement two numerical approaches—**grid search** and **gradient descent** to minimize **root mean squared error (RMSE)**.

We do this in two ways:

1. **Constant-only prediction:** choose a single constant `c` that best predicts the response.
2. **Simple linear regression (SLR):** choose `(b0, b1)` in the prediction rule `y_hat = b0 + b1*x`.


**Dataset:** UCI Air Quality (id = 360)  
**Response variable:** `C6H6(GT)` (benzene concentration)  
**Predictor used for SLR:** `PT08.S1(CO)` (CO sensor signal)



## Setup


We’ll fetch the data using `ucimlrepo` (UCI dataset id = 360).

In [ ]:
!pip -q install ucimlrepo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ucimlrepo import fetch_ucirepo
from scipy.stats import linregress

## Load data

We load the UCI Air Quality dataset (id = 360).  
The `ucimlrepo` object stores the feature table in `dataset.data.features`.

In [ ]:
air_data = fetch_ucirepo(id=360)
df = air_data.data.features.copy()
df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


## Data cleaning

The dataset uses `-200` as a missing-value code.

I convert the table to numeric so the math works consistently, then remove rows where either:
- `C6H6(GT) == -200` (missing benzene), or
- `CO(GT) == -200` (missing CO)



In [ ]:

df_num = df.apply(pd.to_numeric, errors="coerce")

df_clean = df_num.loc[
    (df_num["C6H6(GT)"] != -200) & (df_num["CO(GT)"] != -200)
].dropna(subset=["C6H6(GT)", "CO(GT)"]).copy()





# Constant-only prediction
## `c` as a constant that minimizes RMSE



After removing rows with the missing-value code (-200), `df_clean` is the dataset we use for all steps below.  



## Loss and objective functions (squared error + RMSE)

We are trying to pick numbers that make our predictions “as close as possible” to the real values.

RMSE is the typical prediction mistake size. We will use it in two ways:
- One fixed number
- A line, Simple Linear Regression



In [ ]:
def rmse(y, yhat):
    """Root mean squared error."""
    y = np.asarray(y, dtype=float)
    yhat = np.asarray(yhat, dtype=float)
    return float(np.sqrt(np.mean((y - yhat) ** 2)))


def rmse_constant(y, c):
    """RMSE when predicting all observations with the constant c."""
    y = np.asarray(y, dtype=float)
    yhat = np.full_like(y, float(c), dtype=float)
    return rmse(y, yhat)


def rmse_slr(y, x, b0, b1):
    """RMSE for SLR predictions yhat = b0 + b1*x."""
    y = np.asarray(y, dtype=float)
    x = np.asarray(x, dtype=float)
    return rmse(y, b0 + b1 * x)

## Grid Search

## Grid search for the best fixed number `c`

We pick a single number `c` and predict that same number for every row.

We will:
- make a list of candidate `c` values to try  
- compute RMSE for each
- choose the `c` with the smallest RMSE


### Create the response vector and build the grid (Q1 → Q3)
- Pull the response variable into a NumPy array
- Compute Q1 and Q3 of this variable
- Build a grid of many `c` values between Q1 and Q3.

In [ ]:
y_c6h6 = df_clean["C6H6(GT)"].to_numpy(dtype=float)

q1, q3 = np.quantile(y_c6h6, [0.25, 0.75])
c_grid = np.linspace(q1, q3, 2001)

print("Q1:", q1)
print("Q3:", q3)
print("Grid size:", len(c_grid))

Q1: 4.6
Q3: 14.3
Grid size: 2001


### Evaluate RMSE across the grid and choose the best `c`
- Compute RMSE for each candidate `c` in the grid,
- Find the smallest RMSE
- report the corresponding best `c`


In [ ]:
rmses = np.array([rmse_constant(y_c6h6, c) for c in c_grid], dtype=float)
best_idx = int(np.argmin(rmses))

best_c = float(c_grid[best_idx])
best_rmse = float(rmses[best_idx])

print("Best c:", best_c)
print("Best RMSE:", best_rmse)

Best c: 10.2745
Best RMSE: 7.440561193644043


### Wrap the constant grid search into a function

This function will:
- take a response vector `y`
- build the Q1–Q3 grid
- compute RMSE for each `c`
- return the best `c`


In [ ]:
def grid_search_constant(y, grid_points=2001):
    """Grid search for best constant c using a Q1→Q3 grid."""
    y = np.asarray(y, dtype=float)
    q1, q3 = np.quantile(y, [0.25, 0.75])
    grid = np.linspace(q1, q3, grid_points)
    rmses = np.array([rmse_constant(y, c) for c in grid], dtype=float)
    j = int(np.argmin(rmses))
    return float(grid[j]), float(rmses[j])

### Run the grid search


In [ ]:
# C6H6(GT)
best_c, best_rmse = grid_search_constant(y_c6h6, grid_points=2001)
print("C6H6(GT) grid search")
print("  best c:", best_c)
print("  best RMSE:", best_rmse)
print("  sample mean:", float(np.mean(y_c6h6)))

# PT08.S1(CO)
y_pt08 = df_clean["PT08.S1(CO)"].to_numpy(dtype=float)
best_c2, best_rmse2 = grid_search_constant(y_pt08, grid_points=2001)
print("\nPT08.S1(CO) grid search")
print("  best c:", best_c2)
print("  best RMSE:", best_rmse2)
print("  sample mean:", float(np.mean(y_pt08)))

C6H6(GT) grid search
  best c: 10.2745
  best RMSE: 7.440561193644043
  sample mean: 10.275735294117647

PT08.S1(CO) grid search
  best c: 1110.55
  best RMSE: 218.6664446896789
  sample mean: 1110.5807461873637


For `C6H6(GT)`, the best single prediction number was 10.2745.  
That’s basically the same as the sample average (10.2757), and the RMSE was about 7.44.

For `PT08.S1(CO)`, the best single prediction number was 1110.55, again extremely close to its sample average (1110.58), with RMSE about 218.67.

So in both cases, the “best one-number guess” ends up being essentially the average.

## Gradient descent

In this approach we will:

- start with a guess for `c`  
- figure out whether RMSE would go up or down if we change `c` slightly  
- move `c` a small amount in the direction that lowers RMSE  
- repeat until we stop changing, or hit a safety limit


## Difference quotient function

This helper function:
- takes the current `c`,
- computes RMSE at `c` and at `c + delta`,
- returns the slope estimate.


In [ ]:
def diff_quotient_constant(y, c, delta):
    return (rmse_constant(y, c + delta) - rmse_constant(y, c)) / delta

### Wrap gradient descent into a function

Next we put the gradient descent steps into a function so we can reuse it for different response columns and different tuning parameters.

In [ ]:
def gradient_descent_constant(
    y, start_c,
    delta=0.001, step_size=0.01,
    tol=1e-4, max_iter=50_000
):
    """Gradient descent for constant c using difference quotients."""
    cur_c = float(start_c)

    for it in range(max_iter):
        slope = diff_quotient_constant(y, cur_c, delta)
        new_c = cur_c - slope * step_size

        if abs(new_c - cur_c) < tol:
            return float(new_c), it + 1, "converged"

        cur_c = float(new_c)

    return float(cur_c), max_iter, "max_iter"

###  Run gradient descent for both variables

Our initial guess are:
- For `C6H6(GT)`: start 0, delta 0.001, step 0.01, tol 0.0001  
- For `PT08.S1(CO)`: start 1100, delta 0.001, step 0.1, tol 0.0001

Then report:
- the final `c` value,
- how many iterations it took,
- RMSE at that final `c`.

In [ ]:
# C6H6(GT)
gd_c, gd_iters, gd_status = gradient_descent_constant(
    y_c6h6, start_c=0.0, delta=0.001, step_size=0.01, tol=1e-4, max_iter=50_000
)
print("C6H6(GT) gradient descent")
print("  status:", gd_status)
print("  c*:", gd_c)
print("  iters:", gd_iters)
print("  sample mean:", float(np.mean(y_c6h6)))
print("  RMSE:", rmse_constant(y_c6h6, gd_c))

# PT08.S1(CO)
gd_c2, gd_iters2, gd_status2 = gradient_descent_constant(
    y_pt08, start_c=1100.0, delta=0.001, step_size=0.1, tol=1e-4, max_iter=100_000
)
print("\nPT08.S1(CO) gradient descent")
print("  status:", gd_status2)
print("  c*:", gd_c2)
print("  iters:", gd_iters2)
print("  sample mean:", float(np.mean(y_pt08)))
print("  RMSE:", rmse_constant(y_pt08, gd_c2))

C6H6(GT) gradient descent
  status: converged
  c*: 10.200993520073336
  iters: 3966
  sample mean: 10.275735294117647
  RMSE: 7.440936478911602

PT08.S1(CO) gradient descent
  status: converged
  c*: 1110.3616956683077
  iters: 8483
  sample mean: 1110.5807461873637
  RMSE: 218.66655224571102




For `C6H6(GT)`, gradient descent converged in 3966 iterations to c = 10.2010.  
That’s close to the sample average (10.2757), and the RMSE (7.4409) is basically the same as the grid search RMSE (7.4406).

For `PT08.S1(CO)`, it converged in 8483 iterations to c = 1110.3617, also close to the average (1110.5807), with RMSE 218.6666 (again very close to grid search).

Overall, grid search and gradient descent land in the same neighborhood. Grid search gets there by trying many values and gradient descent gets there by taking small steps.


# Simple Linear Regression (SLR)



## 6. Grid search for SLR parameters `(b0, b1)`

In:
```text
predicted_y = b0 + b1 [link text](https://)* x
```

b0 is the starting level (what the line predicts when x = 0)

b1 tells how much the prediction changes when x increases by 1

### Create x and y arrays + coefficient grids

In [ ]:
x = df_clean["PT08.S1(CO)"].to_numpy(dtype=float)
y = df_clean["C6H6(GT)"].to_numpy(dtype=float)

b0_grid = np.arange(-25, -15 + 1e-12, 0.1)
b1_grid = np.arange(-5, 5 + 1e-12, 0.01)

print("b0 grid size:", len(b0_grid))
print("b1 grid size:", len(b1_grid))

b0 grid size: 101
b1 grid size: 1001


### Implement an efficient SLR grid search function

Next we build a function to:
- pull `x` and `y` into NumPy arrays,
- build the `b0` grid and `b1` grid using given ranges.

In [ ]:
def grid_search_slr(
    y, x,
    b0_grid=None, b1_grid=None,
    b0_start=-25.0, b0_stop=-15.0, b0_step=0.1,
    b1_start=-5.0, b1_stop=5.0, b1_step=0.01
):
    """Grid search for (b0,b1) minimizing RMSE for yhat=b0+b1*x."""
    y = np.asarray(y, dtype=float)
    x = np.asarray(x, dtype=float)

    if b0_grid is None:
        b0_grid = np.arange(b0_start, b0_stop + 1e-12, b0_step, dtype=float)
    else:
        b0_grid = np.asarray(b0_grid, dtype=float)

    if b1_grid is None:
        b1_grid = np.arange(b1_start, b1_stop + 1e-12, b1_step, dtype=float)
    else:
        b1_grid = np.asarray(b1_grid, dtype=float)

    best_rmse = np.inf
    best_b0 = float("nan")
    best_b1 = float("nan")

    x_col = x.reshape(-1, 1)
    y_col = y.reshape(-1, 1)

    for b0 in b0_grid:
        preds = b0 + x_col * b1_grid.reshape(1, -1)           # shape (n, m)
        rmses = np.sqrt(np.mean((y_col - preds) ** 2, axis=0)) # shape (m,)
        j = int(np.argmin(rmses))

        if rmses[j] < best_rmse:
            best_rmse = float(rmses[j])
            best_b0 = float(b0)
            best_b1 = float(b1_grid[j])

    return best_b0, best_b1, best_rmse

### Run the SLR grid search to find the best (b0, b1) and RMSE for the pair

In [ ]:
b0_hat, b1_hat, rmse_hat = grid_search_slr(y, x, b0_grid=b0_grid, b1_grid=b1_grid)

print("SLR grid search result")
print("  b0*:", b0_hat)
print("  b1*:", b1_hat)
print("  RMSE:", rmse_hat)

SLR grid search result
  b0*: -22.99999999999997
  b1*: 0.02999999999989278
  RMSE: 3.542905390830673


### Interpretation (SLR grid search vs direct regression)

Grid search found b0 = -23.0 and b1 = 0.03, with RMSE 3.5429.

The direct regression calculation / `linregress` gave b0 = -22.9557 and b1 = 0.02992, with RMSE 3.5426.

These are very close. The small difference is expected since grid search only checks coefficients at fixed step sizes.
Also, using the line improves the typical error a lot: RMSE drops from about 7.44 down to about 3.54 with the line.


In [ ]:
# Closed-form estimates
xbar = np.mean(x)
ybar = np.mean(y)

b1_closed = np.sum((x - xbar) * (y - ybar)) / np.sum((x - xbar) ** 2)
b0_closed = ybar - xbar * b1_closed

# scipy.stats
res = linregress(x, y)
b1_scipy = res.slope
b0_scipy = res.intercept

print("Closed-form estimates:")
print("  b0:", b0_closed)
print("  b1:", b1_closed)

print("\nscipy.stats.linregress estimates:")
print("  b0:", b0_scipy)
print("  b1:", b1_scipy)

print("\nAbsolute differences (closed-form vs scipy):")
print("  |b0_closed - b0_scipy| =", abs(b0_closed - b0_scipy))
print("  |b1_closed - b1_scipy| =", abs(b1_closed - b1_scipy))

print("\nRMSE comparisons:")
print("  RMSE(closed-form) =", rmse_slr(y, x, b0_closed, b1_closed))
print("  RMSE(scipy)       =", rmse_slr(y, x, b0_scipy, b1_scipy))
print("  RMSE(grid search) =", rmse_slr(y, x, b0_hat, b1_hat))



Closed-form estimates:
  b0: -22.95574738007751
  b1: 0.02992261732276488

scipy.stats.linregress estimates:
  b0: -22.955747380077504
  b1: 0.029922617322764876

Absolute differences (closed-form vs scipy):
  |b0_closed - b0_scipy| = 7.105427357601002e-15
  |b1_closed - b1_scipy| = 3.469446951953614e-18

RMSE comparisons:
  RMSE(closed-form) = 3.542619719180519
  RMSE(scipy)       = 3.5426197191805193
  RMSE(grid search) = 3.5429053908306782


The slope is about 0.03, so when `PT08.S1(CO)` goes up by 100, the predicted `C6H6(GT)` goes up by about 3.

Higher sensor readings produce higher predicted benzene values.


## Predict new C6H6(GT) values using grid-search coefficients

- `PT08.S1(CO)` = 946, 1075, 1246

We plug each x value into:
`predicted_y = b0 + b1 * x`
using our best grid-search `b0` and `b1`.

In [ ]:
new_x = np.array([946, 1075, 1246], dtype=float)
preds_grid = b0_hat + b1_hat * new_x

pd.DataFrame({
    "PT08.S1(CO)": new_x,
    "Predicted C6H6(GT) (grid search)": preds_grid
})

,PT08.S1(CO),Predicted C6H6(GT) (grid search)
0,946.0,5.38
1,1075.0,9.25
2,1246.0,14.38


.. need to keep working on this

### Next we write two helper functions:
- one that estimates the slope “in the b0 direction”
- one that estimates the slope “in the b1 direction”

Each one uses the same idea as before: compare RMSE at the current value vs RMSE after a nudging it.

In [ ]:
def diff_quotient_b0(y, x, b0, b1, delta0):
    return (rmse_slr(y, x, b0 + delta0, b1) - rmse_slr(y, x, b0, b1)) / delta0

def diff_quotient_b1(y, x, b0, b1, delta1):
    return (rmse_slr(y, x, b0, b1 + delta1) - rmse_slr(y, x, b0, b1)) / delta1

### Wrap the line GD algorithm into a function

Next we wrap the full two-parameter gradient descent process into a function.

The function will:
- start at initial `(b0, b1)`,
- update `b0` then `b1` repeatedly,
- stop when the step distance is below tolerance.

In [ ]:
def gradient_descent_slr(
    y, x,
    start_b0=-20.0, start_b1=0.0,
    delta0=0.005, delta1=0.005,
    step_b0=0.5, step_b1=0.00005,
    tol=1e-4, max_iter=100_000
):
    """Gradient descent for SLR using difference quotients."""
    y = np.asarray(y, dtype=float)
    x = np.asarray(x, dtype=float)

    cur_b0 = float(start_b0)
    cur_b1 = float(start_b1)

    for it in range(max_iter):
        slope0 = diff_quotient_b0(y, x, cur_b0, cur_b1, delta0)
        new_b0 = cur_b0 - slope0 * step_b0

        slope1 = diff_quotient_b1(y, x, new_b0, cur_b1, delta1)
        new_b1 = cur_b1 - slope1 * step_b1

        step_dist = float(np.linalg.norm([new_b0 - cur_b0, new_b1 - cur_b1]))

        cur_b0, cur_b1 = float(new_b0), float(new_b1)

        if step_dist < tol:
            return cur_b0, cur_b1, it + 1, "converged"

    return cur_b0, cur_b1, max_iter, "max_iter"

### Run the line GD algorithm and compare to the usual regression solution


We report:
- whether we stopped due to tolerance or due to max_iter
- the final `(b0, b1)` values
- RMSE for those values
- and how they compare to the usual regression (from `linregress`)


In [ ]:
gd_b0, gd_b1, gd_iters, gd_status = gradient_descent_slr(
    y, x,
    start_b0=-20.0, start_b1=0.0,
    delta0=0.005, delta1=0.005,
    step_b0=0.5, step_b1=0.00005,
    tol=1e-4, max_iter=100_000
)

print("SLR gradient descent result")
print("  status:", gd_status)
print("  b0:", gd_b0)
print("  b1:", gd_b1)
print("  iters:", gd_iters)
print("  RMSE:", rmse_slr(y, x, gd_b0, gd_b1))

print("\nReference coefficients")
print("  closed-form:", (b0_closed, b1_closed))
print("  scipy linregress    :", (b0_scipy, b1_scipy))
print("  grid search         :", (b0_hat, b1_hat))

SLR gradient descent result
  status: max_iter
  b0: -22.304829758047333
  b1: -0.0014903666813404631
  iters: 100000
  RMSE: 35.097272964378526

Reference coefficients
  closed-form: (np.float64(-22.95574738007751), np.float64(0.02992261732276488))
  scipy linregress    : (np.float64(-22.955747380077504), np.float64(0.029922617322764876))
  grid search         : (-22.99999999999997, 0.02999999999989278)


### Predict new C6H6(GT) values using gradient descent coefficients

Below are the same three predictions as before (x = 946, 1075, 1246), but using the `(b0, b1)` values from gradient descent.


 I need to keep working on this part...


In [ ]:
preds_gd = gd_b0 + gd_b1 * new_x

pd.DataFrame({
    "PT08.S1(CO)": new_x,
    "Predicted C6H6(GT) (gradient descent)": preds_gd
})

,PT08.S1(CO),Predicted C6H6(GT) (gradient descent)
0,946.0,-23.714717
1,1075.0,-23.906974
2,1246.0,-24.161827


## Wrap-up

...

